No Place for Hate

We seek to find the correlation of the occurrence of Hate Crimes and:
* Education
* Poverty
* Unemployment Rates




In [29]:
#Import Dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import linregress


In [41]:
# import and merge data sets - Education, and Unemployment Rates by state
#read Education csv file and review the sample rows
edu_data = pd.read_csv("Resources/EducationReport.csv")
edu_data.head(3)

,Unnamed: 0,Total,Total.1,Total.2,Total.3,Total.4,Urban,Urban.1,Urban.2,Urban.3,Urban.4,Rural,Rural.1,Rural.2,Rural.3,Rural.4
0,Name,1970,1980,1990,2000,2014-2018,1970,1980,1990,2000,2014-2018,1970,1980,1990,2000,2014-2018
1,United States,10.70%,16.20%,20.30%,24.40%,31.50%,11.50%,17.50%,22.00%,26.20%,33.50%,6.70%,10.50%,12.30%,14.90%,19.60%
2,Alabama,7.80%,12.20%,15.70%,19.00%,24.90%,8.90%,13.70%,17.80%,21.70%,28.00%,5.30%,8.20%,9.80%,11.60%,15.20%


In [43]:
#create a new dataframe with only needed columns for years 2014-2018
edu_data_df = pd.DataFrame(edu_data)
edu_data_mycolm = edu_data_df[["Unnamed: 0", "Total.4", "Urban.4", "Rural.4"]]
edu_data_upd = edu_data_mycolm.rename(columns={"Unnamed: 0" : "State Name", "Total.4" : "Total", "Urban.4" : "Total Urban", "Rural.4" : "Total Rural"})
edu_data_upd

,State Name,Total,Total Urban,Total Rural
0,Name,2014-2018,2014-2018,2014-2018
1,United States,31.50%,33.50%,19.60%
2,Alabama,24.90%,28.00%,15.20%
3,Alaska,29.20%,31.80%,23.90%
4,Arizona,28.90%,29.60%,15.80%
5,Arkansas,22.60%,26.80%,16.00%
6,California,33.30%,33.50%,23.20%
7,Colorado,40.10%,41.40%,31.80%
8,Connecticut,38.90%,39.20%,35.10%
9,Delaware,31.40%,31.40%,NaN


In [10]:
unempl_data = pd.read_csv("Resources/Unemployment.csv", encoding="ISO-8859-1")
unempl_data.head(5)

,FIPStxt,Stabr,area_name,Rural_urban_continuum_code_2013,Urban_influence_code_2013,Metro_2013,Civilian_labor_force_2000,Employed_2000,Unemployed_2000,Unemployment_rate_2000,...,Civilian_labor_force_2018,Employed_2018,Unemployed_2018,Unemployment_rate_2018,Civilian_labor_force_2019,Employed_2019,Unemployed_2019,Unemployment_rate_2019,Median_Household_Income_2018,Med_HH_Income_Percent_of_State_Total_2018
0,0,US,United States,NaN,NaN,NaN,"142,601,667","136,904,680","5,696,987",4.0,...,"161,389,026","155,102,319","6,286,707",3.9,"163,100,055","157,115,247","5,984,808",3.7,"61,937",NaN
1,1000,AL,Alabama,NaN,NaN,NaN,"2,133,223","2,035,594","97,629",4.6,...,"2,216,627","2,130,845","85,782",3.9,"2,241,747","2,174,483","67,264",3.0,"49,881",100.0
2,1001,AL,"Autauga County, AL",2.0,2.0,1.0,"21,720","20,846",874,4.0,...,"26,196","25,261",935,3.6,"26,172","25,458",714,2.7,"59,338",119.0
3,1003,AL,"Baldwin County, AL",3.0,2.0,1.0,"69,533","66,971","2,562",3.7,...,"95,233","91,809","3,424",3.6,"97,328","94,675","2,653",2.7,"57,588",115.5
4,1005,AL,"Barbour County, AL",6.0,6.0,0.0,"11,373","10,748",625,5.5,...,"8,414","7,987",427,5.1,"8,537","8,213",324,3.8,"34,382",68.9


In [ ]:
# Merge the merged Demographic Data with Hate Crimes 358 by state

In [ ]:
# occurrence of Hate Crimes to Education (Bachelor's Degree vs. High School) by State

In [ ]:
# occurrence of Hate Crimes to Poverty Level by State

In [ ]:
# occurrence of Hate Crimes to Poverty Level by State

In [ ]:
# Clean FBI Data set - take 2014-2018 data and average hate crimes fbi